<a href="https://colab.research.google.com/github/jwtan0421/LLaMA-Factory-FT/blob/main/Finetune_with_LLaMA_Factory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fine tune using Qwen 2.5 - 7B

In [ ]:
!pip install autoawq

In [ ]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
!pip uninstall -y jax
!pip install -e .[torch,bitsandbytes,liger-kernel]

In [3]:
%cd /content/
!git clone https://huggingface.co/Qwen/Qwen2.5-7B-Instruct

/content
Cloning into 'Qwen2.5-7B-Instruct'...
remote: Enumerating objects: 47, done.
remote: Total 47 (delta 0), reused 0 (delta 0), pack-reused 47 (from 1)
Unpacking objects: 100% (47/47), 3.61 MiB | 8.27 MiB/s, done.
Filtering content: 100% (4/4), 14.18 GiB | 101.88 MiB/s, done.


In [4]:
%cd /content/
!git clone https://github.com/jwtan0421/LLaMA-Factory-FT.git

/content
Cloning into 'LLaMA-Factory-FT'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 6 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), done.
Resolving deltas: 100% (1/1), done.


In [5]:
import json

%cd /content/LLaMA-Factory/

NAME = "QWen-2.5"
AUTHOR = "Osborn"

with open("data/identity.json", "r", encoding="utf-8") as f:
  dataset = json.load(f)

for sample in dataset:
  sample["output"] = sample["output"].replace("{{"+ "name" + "}}", NAME).replace("{{"+ "author" + "}}", AUTHOR)

with open("data/identity.json", "w", encoding="utf-8") as f:
  json.dump(dataset, f, indent=2, ensure_ascii=False)

/content/LLaMA-Factory


In [6]:
!cp /content/LLaMA-Factory-FT/train_qlora/qwen_lora_sft_bitsandbytes.yaml /content/LLaMA-Factory/examples/train_qlora

In [8]:
# trainig
%cd /content/LLaMA-Factory/

!llamafactory-cli train examples/train_qlora/qwen_lora_sft_bitsandbytes.yaml

/content/LLaMA-Factory
2024-11-27 05:49:58.203179: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 05:49:58.220186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 05:49:58.241006: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 05:49:58.247424: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 05:49:58.262404: I tensorflow/core/pl

In [9]:
!cp /content/LLaMA-Factory-FT/inference/qwen_lora_sft.yaml /content/LLaMA-Factory/examples/inference

In [12]:
# inference
!llamafactory-cli chat examples/inference/qwen_lora_sft.yaml

2024-11-27 06:01:33.777708: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 06:01:33.794810: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 06:01:33.815608: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 06:01:33.821955: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 06:01:33.837186: I tensorflow/core/platform/cpu_feature_guar

### Fine tune using Llama 3.2 - 3B

In [14]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `Colab` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pus

In [20]:
# trainig
%cd /content/LLaMA-Factory/

!llamafactory-cli train examples/train_qlora/llama3.2_lora_sft_awq.yaml

/content/LLaMA-Factory
2024-11-27 09:02:10.464341: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 09:02:10.481657: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 09:02:10.502900: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 09:02:10.509342: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 09:02:10.524543: I tensorflow/core/pl

In [23]:
# inference
%cd /content/LLaMA-Factory/

!llamafactory-cli chat examples/inference/llama3.2_lora_sft.yaml

/content/LLaMA-Factory
2024-11-27 09:18:58.800533: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 09:18:58.817728: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 09:18:58.838814: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 09:18:58.845232: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 09:18:58.860500: I tensorflow/core/pl

/content/LLaMA-Factory/saves/llama3.2-8b/lora/sft